In [250]:
from sklearn import preprocessing 
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor

import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.options.mode.chained_assignment = None

In [251]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv('gender_submission.csv')

In [252]:
data_combine = train.append(test)
data_combine.reset_index(inplace=True, drop=True)
data_combine.shape

(1309, 12)

## Feature engineering

In [253]:
# combine SibSp and Parch
data_combine['family'] = data_combine['SibSp'] + data_combine['Parch']
data_combine.drop('SibSp', 1, inplace=True)
data_combine.drop('Parch', 1, inplace=True)
data_combine.shape

(1309, 11)

In [254]:
data_combine['Ticket_info'] = \
data_combine['Ticket'].apply(lambda x : x.replace(".","").replace("/","").strip().split(' ')[0] if not x.isdigit() else 'X')

In [255]:
# Fare fillna with mean
data_combine['Fare'] = data_combine['Fare'].fillna(data_combine['Fare'].mean())

In [256]:
# Embarked fillna with C
data_combine['Embarked'] = data_combine['Embarked'].fillna("C")

In [257]:
# Cabin fillna
'''
mask = (data_combine["Survived"]==0.0) & (data_combine["Cabin"].isnull())
data_combine.loc[mask, "Cabin"] = "G"

mask = (data_combine["Survived"]==1.0) & (data_combine["Cabin"].isnull())
data_combine.loc[mask, "Cabin"] = "B"

mask = (data_combine["Survived"].isnull()) & (data_combine["Cabin"].isnull()) & (data_combine["Pclass"] == 1)
data_combine.loc[mask, "Cabin"] = "B"

mask = (data_combine["Survived"].isnull()) & (data_combine["Cabin"].isnull()) & (data_combine["Pclass"] == 2)
data_combine.loc[mask, "Cabin"] = "F"

mask = (data_combine["Survived"].isnull()) & (data_combine["Cabin"].isnull()) & (data_combine["Pclass"] == 3)
data_combine.loc[mask, "Cabin"] = "G"

data_combine["Cabin"] = data_combine['Cabin'].apply(lambda x : str(x)[0])
'''
data_combine["Cabin"] = data_combine['Cabin'].apply(lambda x : str(x)[0] if not pd.isnull(x) else 'NoCabin')
# data_combine['Cabin']

In [258]:
# Name
'''
for typeName in ["Ms.", "Miss.", "Mr.", "Master.", "Mme."]:
   
    mask =  (data_combine['Name'].str.contains(typeName))
    replaceTitle = typeName
        
    if(typeName=="Mme."):
        replaceTitle = "Ms."
        
    data_combine.loc[mask,'Title'] = replaceTitle
    
mask = data_combine["Title"].isnull() & (data_combine['Age']<14) & (data_combine['Sex']=='male')
data_combine.loc[mask, "Title"] = "Master."

mask = data_combine["Title"].isnull() & (data_combine['Age']>13) & (data_combine['Sex']=='male')
data_combine.loc[mask, "Title"] = "Mr."

mask = data_combine["Title"].isnull() & (data_combine['Sex']=='female')
data_combine.loc[mask, "Title"] = "Miss."

data_combine.drop('Name',axis=1,inplace=True)
'''
data_combine['Title1'] = data_combine['Name'].str.split(", ", expand=True)[1]
data_combine['Title1'] = data_combine['Title1'].str.split(".", expand=True)[0]
data_combine['Title'] = data_combine['Title1'] \
.replace(['Mlle','Mme','Ms','Dr','Major','Lady','the Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],
         ['Miss','Mrs','Miss','Mr','Mr','Mrs','Mrs','Mr','Mr','Mr','Mr','Mr','Mr','Mrs'])

In [259]:
# Label Encoding : http://pbpython.com/categorical-encoding.html
data_combine['Sex'] = data_combine['Sex'].astype('category').cat.codes
data_combine['Embarked'] = data_combine['Embarked'].astype('category').cat.codes
data_combine['Pclass'] = data_combine['Pclass'].astype('category').cat.codes
data_combine['Title'] = data_combine['Title'].astype('category').cat.codes
data_combine['Cabin'] = data_combine['Cabin'].astype('category').cat.codes
data_combine['Ticket_info'] = data_combine['Ticket_info'].astype('category').cat.codes
data_combine.head(5)

,Age,Cabin,Embarked,Fare,Name,PassengerId,Pclass,Sex,Survived,Ticket,family,Ticket_info,Title1,Title
0,22.0,7,2,7.2500,"Braund, Mr. Owen Harris",1,2,1,0.0,A/5 21171,1,2,Mr,2
1,38.0,2,0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",2,0,0,1.0,PC 17599,1,14,Mrs,3
2,26.0,7,2,7.9250,"Heikkinen, Miss. Laina",3,2,0,1.0,STON/O2. 3101282,0,31,Miss,1
3,35.0,2,2,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",4,0,0,1.0,113803,1,36,Mrs,3
4,35.0,7,2,8.0500,"Allen, Mr. William Henry",5,2,1,0.0,373450,0,36,Mr,2


In [260]:
# Age
dataAgeNull = data_combine[data_combine["Age"].isnull()]
dataAgeNotNull = data_combine[data_combine["Age"].notnull()]

remove_outlier = \
dataAgeNotNull[~((np.abs(dataAgeNotNull["Fare"]-dataAgeNotNull["Fare"].mean())>(4*dataAgeNotNull["Fare"].std()))|
(np.abs(dataAgeNotNull["Ticket_info"]-dataAgeNotNull["Ticket_info"].mean())>(4*dataAgeNotNull["Ticket_info"].std())))]


rfModel_age = RandomForestRegressor(n_estimators=2000,random_state=42)

ageColumns = ['Embarked', 'Fare', 'Pclass', 'Sex', 'family', 'Title','Cabin','Ticket_info']
rfModel_age.fit(remove_outlier[ageColumns], remove_outlier["Age"])

ageNullValues = rfModel_age.predict(X= dataAgeNull[ageColumns])
dataAgeNull.loc[:,"Age"] = ageNullValues
data_combine = dataAgeNull.append(dataAgeNotNull)
data_combine.reset_index(inplace=True, drop=True)

#print(data_combine.describe())

In [261]:
# train
dataTrain = data_combine[data_combine['Survived'].notnull()].sort_values(by=["PassengerId"])
dataTest = data_combine[data_combine['Survived'].isnull()].sort_values(by=["PassengerId"])

dataTrain = dataTrain[['Survived', 'Age', 'Embarked', 'Fare',  'Pclass', 'Sex', 'family', 'Title','Ticket_info','Cabin']]
dataTest = dataTest[['Age', 'Embarked', 'Fare', 'Pclass', 'Sex', 'family', 'Title','Ticket_info','Cabin']]

In [262]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics, cross_validation
 
rf = RandomForestClassifier(criterion='gini', 
                             n_estimators=2000,
                             min_samples_split=12,
                             min_samples_leaf=1,
                             oob_score=True,
                             random_state=4,
                             n_jobs=-1) 

rf.fit(dataTrain.iloc[:, 1:], dataTrain.iloc[:, 0])

print("%.4f" % rf.oob_score_)   #maybe he higher the beter

0.8328


In [263]:
rf_res =  rf.predict(dataTest)
print(rf_res.shape)
print(submit['Survived'].shape)      

submit['Survived'] = submit['Survived'].astype(int)
submit.to_csv('submit.csv', index= False)

(418,)
(418,)
